# Generate QAs from Unstructured Data Using AWS Inferentia and Neuron

Are you a machine learning (ML) professional grappling with the challenge of extracting high-quality data from confidential and unstructured formats such as PDFs and HTML? The task of fine-tuning large language models (LLMs) often requires dealing with such complex data, which poses significant challenges in terms of data privacy, processing efficiency, and model accuracy.
In this example, we will show you how to generate question-answers (QAs) from give text strings using open-source models via uniflow's [HuggingFaceModelFlow](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/transform/transform_huggingface_flow.py).

This article provides a comprehensive guide on leveraging the power of Uniflow on [Amazon Web Services (AWS) Inferentia2 (Inf2) instances](https://aws.amazon.com/machine-learning/inferentia/) for effectively converting unstructured data into a structured format, ideal for private LLM fine-tuning. We delve into the intricacies of handling sensitive information while maintaining confidentiality and data integrity.  Furthermore, we explore the robust capabilities of the AWS Neuron SDK, which plays a pivotal role in enhancing the performance of your ML models. The Neuron SDK is specifically designed to optimize ML workloads on AWS hardware, ensuring you achieve faster processing speeds and reduced operational costs.

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

### Update system path

In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

### Install libraries

In [2]:
!{sys.executable} -m pip install -q transformers accelerate bitsandbytes scipy

### Import dependency

In [ ]:
from uniflow.flow.client import TransformClient
from uniflow.flow.config import TransformHuggingFaceConfig, HuggingfaceModelConfig
from uniflow.op.prompt import PromptTemplate, Context
import time

### Prepare sample prompts

First, we need to demonstrate sample prompts for LLM, those include instruction and sample json format. We do this by giving a sample instruction and list of `Context` examples to the `PromptTemplate` class.

In [4]:
sample_instruction = """Generate one question and its corresponding answer based on the context. Following \
the format of the examples below to include context, question, and answer in the response."""

sample_examples = [
        Context(
            context="The quick brown fox jumps over the lazy dog.",
            question="What is the color of the fox?",
            answer="brown."
        ),
        Context(
            context="The quick brown fox jumps over the lazy black dog.",
            question="What is the color of the dog?",
            answer="black."
        )]

guided_prompt = PromptTemplate(
    instruction=sample_instruction,
    few_shot_prompt=sample_examples
)

Second, we craft some dummy sample raw text strings. Below, we build a dataset with 400 text strings.

In [5]:
raw_context_input = [
    """We believe our success depends upon our capabilities in areas such as design, research and development, \
production and marketing and is supported and protected by our intellectual property rights, such as \
trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy \
of applying for and registering intellectual property rights in the United States and select foreign countries \
on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously \
protect our intellectual property, including trademarks, patents and trade secrets against third-party \
infringement and misappropriation.""",
    """In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948) \
establishing the theory of information. In his article, Shannon introduced the concept of information entropy \
for the first time. We will begin our journey here.""",
    """The chain rule states that the derivative of a composite function (a function composed of another \
function) is equal to the derivative of the outer function multiplied by the derivative of the inner function.\
Mathematically, it can be written as: \(\frac{d}{dx}g(h(x)) = \frac{dg}{dh}(h(x))\cdot \frac{dh}{dx}(x)\).""",
    """Hypothesis testing involves making a claim about a population parameter based on sample data, and then \
conducting a test to determine whether this claim is supported or rejected. This typically involves \
calculating a test statistic, determining a significance level, and comparing the calculated value to a \
critical value to obtain a p-value. """
]

raw_context_input = raw_context_input * 250

print("sample size of raw context: ", len(raw_context_input))

sample size of raw context:  1000


Next, for the given raw text strings `raw_context_input` above, we convert them to the `Context` class to be processed by `uniflow`.

In [6]:

input_data = [
    Context(context=data)
    for data in raw_context_input
]

print("sample size of processed input data: ", len(input_data))

input_data[:2]


sample size of processed input data:  1000


[Context(context='We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in the United States and select foreign countries on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously protect our intellectual property, including trademarks, patents and trade secrets against third-party infringement and misappropriation.'),
 Context(context='In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948) establishing the theory of information. In his article, Shannon introduced the concept of information entropy for the first time. We will begin our journey here.')]

### Use LLM to generate data

In this example, we will use the [HuggingfaceModelConfig](https://github.com/CambioML/uniflow/blob/main/uniflow/model/config.py#L39)'s default LLM to generate questions and answers. Let's import the config and client of this model.

Here, we pass in our `guided_prompt` to the `HuggingfaceConfig` to use our customized instructions and examples, instead of the `uniflow` default ones.

Note, base on your GPU memory, you can set your optimal `batch_size` below. (We attached our `batch_size` benchmarking results in the appendix of this notebook.)

In [7]:
config = TransformHuggingFaceConfig(
    prompt_template=guided_prompt,
    model_config=HuggingfaceModelConfig(batch_size=8, model_name='mistralai/Mistral-7B-Instruct-v0.2', neuron=True))
client = TransformClient(config)

Neuron model does not support quantized models. load_in_4bit and load_in_8bit are automatically set to False.
Spliting model, need to wait for a while...


config.json: 100%|██████████| 596/596 [00:00<00:00, 4.32MB/s]
model.safetensors.index.json: 100%|██████████| 25.1k/25.1k [00:00<00:00, 80.0MB/s]
model-00001-of-00003.safetensors: 100%|██████████| 4.94G/4.94G [00:20<00:00, 239MB/s]
model-00002-of-00003.safetensors: 100%|██████████| 5.00G/5.00G [00:20<00:00, 243MB/s]
model-00003-of-00003.safetensors: 100%|██████████| 4.54G/4.54G [00:17<00:00, 265MB/s]
generation_config.json: 100%|██████████| 111/111 [00:00<00:00, 899kB/s]
100%|██████████| 161M/161M [00:01<00:00, 100MiB/s] 


/home/ubuntu/.cache/uniflow/10b481c5b2f1e3054ca092027443b1a2.zip /home/ubuntu/.cache/uniflow/10b481c5b2f1e3054ca092027443b1a2
loading neuron model from cache, need to wait for a while...


tokenizer_config.json: 100%|██████████| 1.46k/1.46k [00:00<00:00, 11.4MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 14.8MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:00<00:00, 12.4MB/s]
special_tokens_map.json: 100%|██████████| 72.0/72.0 [00:00<00:00, 699kB/s]


Now we call the `run` method on the `client` object to execute the question-answer generation operation on the data shown above.

In [8]:
start_time = time.time()
output = client.run(input_data)
execution_time = time.time() - start_time

  0%|          | 0/125 [00:00<?, ?it/s]

2024-Feb-27 06:44:35.0604 2332:2749 [1] nccl_net_ofi_init:1415 CCOM WARN NET/OFI aws-ofi-nccl initialization failed
2024-Feb-27 06:44:35.0604 2332:2749 [1] init.cc:137 CCOM WARN OFI plugin initNet() failed is EFA enabled?


100%|██████████| 125/125 [10:14<00:00,  4.92s/it]


### Process the output

Let's take a look of the generated output. We need to do a little postprocessing on the raw output.

In [9]:
import re
from pprint import pprint

keywords = ["context:", "question:", "answer:"]
pattern = '|'.join(map(re.escape, keywords))

o = output[0]['output'][0]['response'][0] ## we only postprocess the first output
segments = [segment for segment in re.split(pattern, o) if segment.strip()]
result = {
    "context": segments[-3],
    "question": segments[-2],
    "answer": segments[-1]
}

pprint(result, sort_dicts=False)

{'context': ' We believe our success depends upon our capabilities in areas '
            'such as design, research and development, production and '
            'marketing and is supported and protected by our intellectual '
            'property rights, such as trademarks, utility and design patents, '
            'copyrights, and trade secrets, among others. We have followed a '
            'policy of applying for and registering intellectual property '
            'rights in the United States and select foreign countries on '
            'trademarks, inventions, innovations and designs that we deem '
            'valuable. W e also continue to vigorously protect our '
            'intellectual property, including trademarks, patents and trade '
            'secrets against third-party infringement and misappropriation. ',
 'question': ' What types of intellectual property does the company protect?\n',
 'answer': ' The company protects various intellectual property rights, '
       

In [10]:
def count_tokens(document):
    ''' 
    Give a document(string), return total number of tokens(words)

    Parameters:
       documents(string): A document

    Returns:
       count(int): Total number of tokens. 1 token = 0.75 word
    '''
    num_words = len(document.split())
    num_tokens = round(num_words/0.75)
    return num_tokens

In [11]:
# Count tokens
input_tokens = sum(count_tokens(d) for d in raw_context_input)
prompt_tokens = count_tokens(config.prompt_template.instruction)
output_tokens = 0
for o in output:
    if "output" in o:
        for response in o["output"]:
            for r in response["response"]:
                segments = [segment for segment in re.split(pattern, r) if segment.strip()]
                result = {
                    "context": segments[-3],
                    "question": segments[-2],
                    "answer": segments[-1]
                }
                output_tokens += count_tokens(result["context"])
                output_tokens += count_tokens(result["question"])
                output_tokens += count_tokens(result["answer"])

tokens = input_tokens + prompt_tokens + output_tokens
print("output tokens: ", output_tokens)
print("input tokens: ", input_tokens)
print("prompt tokens: ",prompt_tokens)
print("total tokens: ", tokens)
print("processed tokens per second: ", round(float(tokens/execution_time)))

output tokens:  131263
input tokens:  76000
prompt tokens:  36
total tokens:  207299
processed tokens per second:  337


Congrats! Your question answers from the given knowledge context are generated!


---


## Appendix

We benchmarked to see the optimal `batch_size` for the `HuggingfaceModelConfig` flow. The answer is "It depends on your data token length, your instance type, your LLM size, etc." In the following experiment, we use an AWS inf2.8xlarge. We still use the above 1000 raw data strings `raw_context_input`.


Here are the results:

- batch_size = 1             
    100%|██████████| 1000/1000 [5:11:19<00:00, 18.68s/it]         
    output tokens:  364189    
    input tokens:  76000    
    prompt tokens:  36   
    total tokens:  440225   
    processed tokens per second:  24   

- batch_size = 2         
    100%|██████████| 500/500 [24:03<00:00,  2.89s/it]         
    output tokens:  131427     
    input tokens:  76000    
    prompt tokens:  36    
    total tokens:  207463    
    processed tokens per second:  144     


- batch_size = 4   
    100%|██████████| 250/250 [17:45<00:00,  4.26s/it]   
    output tokens:  132261   
    input tokens:  76000   
    prompt tokens:  36   
    total tokens:  208297   
    processed tokens per second:  195   


- batch_size = 8   
    100%|██████████| 125/125 [10:14<00:00,  4.92s/it]         
    output tokens:  131263   
    input tokens:  76000   
    prompt tokens:  36   
    total tokens:  207299   
    processed tokens per second:  337   



## End of the notebook

Check more Uniflow use cases in the [example folder](https://github.com/CambioML/uniflow/tree/main/example)!

<a href="https://www.cambioml.com/" title="Title">
    <img src="../image/cambioml_logo_large.png" style="height: 100px; display: block; margin-left: auto; margin-right: auto;"/>
</a>